In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from herbie import Herbie, FastHerbie

In [2]:
INFILE_NAME = ""
DATE_FORMAT = r"%Y-%m-%d"
TIME_FORMAT = r"%H:%M:%S"
HRRR_CUTOFF_DATE = datetime.strptime("2014-07-30", DATE_FORMAT)  # earlier than this => use ECMWF

In [3]:
df = pd.DataFrame({})

desired_weather_params = [
    "main.temp",
    "main.feels_like",
    "main.pressure",
    "main.humidity",
    "main.dew_point",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "rain.3h",
    "snow.3h",
    "visibility",
]

for param in desired_weather_params:
    df[param] = np.nan

In [22]:
df = pd.DataFrame({"FID": ["1341515", "13"], "date": ["2019-07-30", "2019-09-30"], "time": ["18:47:13", "13:52:11"]})

MAX_THREADS = 100
HRRR_SRC = "hrrr"
ECMWF_SRC = "ifs"

cases = {HRRR_SRC: [], ECMWF_SRC: []}
for idx, row in df.iterrows():
    date = datetime.strptime(row.at["date"], DATE_FORMAT)

    source = HRRR_SRC if date >= HRRR_CUTOFF_DATE else ECMWF_SRC

    time = datetime.strptime(row.at["time"], TIME_FORMAT)

    cases[source].append((idx, datetime.combine(date.date(), time.time())))


# the input to FastHerbie needs to maintain it's index in the output
# if the result is sorted by time, we need to pre-sort cases by time
FH_HRRR = FastHerbie([c[1] for c in cases[HRRR_SRC]], model=HRRR_SRC, product="sfc", fxx=[0], max_threads=MAX_THREADS)
print(FH_HRRR.xarray(searchString=":TMP:2 m"))
# H_ECMWF = FastHerbie(list(cases[ECMWF_SRC].values()), model=ECMWF_SRC, product="sfc", fxx=0, max_threads=MAX_THREADS)

# for idx, value in enumerate(FH_HRRR):
#     convert_to_api_params(value)
#     for param in params:
#         df.iat[i, param] = param_value

👨🏻‍🏭 Created directory: [/home/charles/data/hrrr/20190730]
👨🏻‍🏭 Created directory: [/home/charles/data/hrrr/20190930]
<xarray.Dataset> Size: 46MB
Dimensions:              (time: 2, y: 1059, x: 1799)
Coordinates:
  * time                 (time) datetime64[ns] 16B 2019-07-30T18:00:00 2019-0...
    step                 timedelta64[ns] 8B 00:00:00
    heightAboveGround    float64 8B 2.0
    latitude             (y, x) float64 15MB 21.14 21.15 21.15 ... 47.85 47.84
    longitude            (y, x) float64 15MB 237.3 237.3 237.3 ... 299.0 299.1
    valid_time           (time) datetime64[ns] 16B 2019-07-30T18:00:00 2019-0...
Dimensions without coordinates: y, x
Data variables:
    t2m                  (time, y, x) float32 15MB 297.3 297.3 ... 280.8 280.8
    gribfile_projection  object 8B None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF